# This chatbot uses vozk for speech to text

In [2]:
import pyaudio

p = pyaudio.PyAudio()

for i in range(p.get_device_count()):
    print(p.get_device_info_by_index(i))
    
p.terminate()

{'index': 0, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Input', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 1, 'structVersion': 2, 'name': 'Varios micrÃ³fonos (Realtek(R) A', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 2, 'structVersion': 2, 'name': 'Mezcla estÃ©reo (Realtek(R) Audi', 'hostApi': 0, 'maxInputChannels': 2, 'maxOutputChannels': 0, 'defaultLowInputLatency': 0.09, 'defaultLowOutputLatency': 0.09, 'defaultHighInputLatency': 0.18, 'defaultHighOutputLatency': 0.18, 'defaultSampleRate': 44100.0}
{'index': 3, 'structVersion': 2, 'name': 'Microsoft Sound Mapper - Output', 'hostApi': 0, 'maxInputCha

In [3]:
CHANNELS = 1
FRAME_RATE = 16000
RECORD_SECONDS = 2
AUDIO_FORMAT = pyaudio.paInt16
SAMPLE_SIZE = 2

MENSAJE = ""

def record_microphone(chunk=1024):
    p = pyaudio.PyAudio()
    stream = p.open(format=AUDIO_FORMAT,
                   channels=CHANNELS,
                   rate=FRAME_RATE,
                   input=True,
                   input_device_index=1,
                   frames_per_buffer=chunk)
    frames = []
    
    while not messages.empty():
        data = stream.read(chunk)
        frames.append(data)
        
        if len(frames) >= (FRAME_RATE * RECORD_SECONDS) / chunk:
            recordings.put(frames.copy())
            frames = []
            
    stream.stop_stream()
    stream.close()
    p.terminate()

In [4]:
import subprocess
import json
from vosk import Model, KaldiRecognizer

model = Model(model_name="vosk-model-es-0.42")
rec = KaldiRecognizer(model, FRAME_RATE)
rec.SetWords(True)


def speech_recognition(output):
    global MENSAJE
    while not messages.empty():
        frames = recordings.get()
        
        rec.AcceptWaveform(b''.join(frames))
        result = rec.Result()
        text = json.loads(result)["text"]
        
        MENSAJE += text + " "
        
        output.append_stdout(text+ " ")
        

In [5]:
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
from queue import Queue

messages = Queue()
recordings = Queue()


record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    global MENSAJE
    MENSAJE = ""
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        
        
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
        
        display("Transcribed Text:", MENSAJE)


record_button.on_click(start_recording)

stop_button.on_click(stop_recording)

display(record_button, stop_button, output)


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

# Now with ChatGPT

In [6]:
from openai import OpenAI
import os

#client.api_key = os.getenv('OPENAI_API_KEY') # asigna la API key de chatGPT

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # lee el archivo .env donde está la api key


client = OpenAI(
    api_key = os.environ.get("OPENAI_API_KEY")
)

In [7]:
def get_completion(prompt):
    completion = client.chat.completions.create(
      model = "gpt-3.5-turbo",
      messages=[
          {"role": "user", "content": prompt}
      ]
    )
    return completion.choices[0].message.content

In [8]:
get_completion("como te llamas?")

'Soy un modelo de lenguaje desarrollado por OpenAI llamado GPT-3. No tengo un nombre específico porque soy una inteligencia artificial. ¿En qué más puedo ayudarte?'

In [9]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [10]:
llm = ChatOpenAI(temperature=0.0)

In [11]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

In [12]:
conversation.predict(input="cual es mi nombre?")

'Lo siento, pero no tengo acceso a tu información personal, por lo que no puedo saber cuál es tu nombre.'

In [13]:
from IPython.display import Audio, display
from pathlib import Path
import time
import openai

memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm,
    memory = memory,
    verbose=False
)

respuesta = ""

prompt = f"""

Eres un sommelier experto en la cata de vino. Tu especialidad es guiar a las personas, sin importar su nivel de conocimiento, a poder descubrir todos los aromas que pueda encontrar en un vino. Para ello, utilizas los siguientes pasos: 
1-	Oler cada muestra en la boca del vaso antes de agitarlo. 
2-	Estudiar y registrar la naturaleza e intensidad del aroma. 
3-	Agitar el vaso para promover la liberación de los componentes aromáticos del vino. 
4-	Oler el vino, primero en la boca y luego más profundamente en la copa. 
5-	 Estudiar y registrar la naturaleza e intensidad del aroma.

Además, toma en cuenta lo siguiente, que existen 3 niveles de aromas en los vinos y quieres ayudar a las personas a encontrarlos

En la enología, existen 3 categorías de aromas. Aromas primarios, es decir, los aromas de la variedad de uva. Cada variedad tiene sus propios aromas únicos y matices vinculados al suelo y al clima. Los aromas secundarios se derivan del proceso de vinificación y de la levadura utilizada para la fermentación (transformación del azúcar natural de la uva en alcohol). Los aromas terciarios surgen durante el proceso de envejecimiento, especialmente cuando el vino se guarda en barricas de madera, en una bodega (también llamado bouquet).

Siguiendo estas estructuras como ejemplo, ahora tú serás el sommelier y estará con un novato en los vinos. Estará degustando un Charles Shaw, 2012 California Cabernet Sauvignon. 
Ahora, escribe la primera línea de lo que de bienvenida que diría el sommelier para identificar los aromas y espera la respuesta del usuario para continuar con la interacción. (El sommelier responde máximo un párrafo corto en cada ocasión). Sommelier:
"""

respuesta = conversation.predict(input=prompt)


palabras = len(respuesta.split())
    
espera = (palabras*60) / 150
print(espera+2)

# Define the path for the output file
speech_file_path = Path("speech.mp3")

# Call the OpenAI API to generate speech
response = openai.audio.speech.create(
    model="tts-1",
    voice="alloy",
    input=respuesta
)

# Stream the response to the specified file
response.stream_to_file(speech_file_path)

# Play the audio in the notebook automatically
audio = Audio(str(speech_file_path), autoplay=True)
display(audio)
#time.sleep(espera)


## AQUI INICIA LO DE VOZK



messages = Queue()
recordings = Queue()


record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    global MENSAJE
    MENSAJE = ""
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()
        
        
def stop_recording(data):
    with output:
        messages.get()
        display("Stopped.")
        display("Transcribed Text:", MENSAJE)
        
        user_input = MENSAJE    
        global respuesta
        respuesta = conversation.predict(input=user_input)
        palabras = len(respuesta.split())
    
        espera = (palabras*60) / 150
        print(espera+2)

        # Define the path for the output file
        speech_file_path = Path("speech.mp3")

        # Call the OpenAI API to generate speech
        response = openai.audio.speech.create(
            model="tts-1",
            voice="alloy",
            input=respuesta
        )

        # Stream the response to the specified file
        response.stream_to_file(speech_file_path)

        # Play the audio in the notebook automatically
        audio = Audio(str(speech_file_path), autoplay=True)
        display(audio)
        #time.sleep(espera)
        


record_button.on_click(start_recording)

stop_button.on_click(stop_recording)

display(record_button, stop_button, output)



18.0


Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()

In [19]:
from IPython.display import Audio, display, clear_output
from pathlib import Path
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
import ipywidgets as widgets
from IPython.display import display
from threading import Thread
import openai
import time
import threading
from queue import Queue
from pydub import AudioSegment
from pydub.playback import play

# Initialize the conversation
llm = ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
conversation = ConversationChain(llm=llm, memory=memory, verbose=False)

# Load the audio file (replace 'loading_sound.mp3' with your audio file path)
loading_sound = AudioSegment.from_file('relaxing.mp3') - 30


# ... (Rest of your code remains the same)
respuesta = ""
current_audio = None

prompt = f"""

Eres un sommelier experto en la cata de vino. Tu especialidad es guiar a las personas, sin importar su nivel de conocimiento, a poder descubrir todos los aromas que pueda encontrar en un vino. Para ello, utilizas los siguientes pasos: 
1-	Oler cada muestra en la boca del vaso antes de agitarlo. 
2-	Estudiar y registrar la naturaleza e intensidad del aroma. 
3-	Agitar el vaso para promover la liberación de los componentes aromáticos del vino. 
4-	Oler el vino, primero en la boca y luego más profundamente en la copa. 
5-	 Estudiar y registrar la naturaleza e intensidad del aroma.

Además, toma en cuenta lo siguiente, que existen 3 niveles de aromas en los vinos y quieres ayudar a las personas a encontrarlos

En la enología, existen 3 categorías de aromas. Aromas primarios, es decir, los aromas de la variedad de uva. Cada variedad tiene sus propios aromas únicos y matices vinculados al suelo y al clima. Los aromas secundarios se derivan del proceso de vinificación y de la levadura utilizada para la fermentación (transformación del azúcar natural de la uva en alcohol). Los aromas terciarios surgen durante el proceso de envejecimiento, especialmente cuando el vino se guarda en barricas de madera, en una bodega (también llamado bouquet).

Siguiendo estas estructuras como ejemplo, ahora tú serás el sommelier y estará con un novato en los vinos. Estará degustando un Charles Shaw, 2012 California Cabernet Sauvignon. 
Ahora, escribe la primera línea de lo que de bienvenida que diría el sommelier para identificar los aromas y espera la respuesta del usuario para continuar con la interacción. (El sommelier responde máximo un párrafo corto en cada ocasión). Sommelier:
"""


# Global flag to control the loading sound playback
stop_loading_sound = False

def play_loading_sound_continuously():
    global stop_loading_sound
    while not stop_loading_sound:
        play(loading_sound)
        time.sleep(0.05)  # Adjust the sleep time as needed


# Define a global variable to store the response text
respuesta = ""
current_audio = None
current_audio_path = None
audio_output = widgets.Output()
display(audio_output)

# Function to generate speech for the response
def generate_speech(response_text):
    global current_audio
    global current_audio_path
    global stop_loading_sound

    palabras = len(response_text.split())
    espera = (palabras * 60) / 150 + 2

    # Define the path for the output file
    speech_file_path = Path("speech.mp3")

    # Call the OpenAI API to generate speech
    response = openai.audio.speech.create(
        model="tts-1",
        voice="alloy",
        input=response_text
    )


    # Stream the response to the specified file
    response.stream_to_file(speech_file_path)
    current_audio_path = str(speech_file_path)
    
    stop_loading_sound = True

    # Clear the previous audio from the audio output widget
    with audio_output:
        clear_output(wait=True)
        display(Audio(str(speech_file_path), autoplay=True))
    
    
    
    
respuesta = conversation.predict(input=prompt)

generate_speech(respuesta)

# ... (Rest of your code remains the same)
messages = Queue()
recordings = Queue()


record_button = widgets.Button(
    description="Record",
    disabled=False,
    button_style="success",
    icon="microphone"
)

stop_button = widgets.Button(
    description="Stop",
    disabled=False,
    button_style="warning",
    icon="stop"
)

output = widgets.Output()

def start_recording(data):
    global MENSAJE
    MENSAJE = ""
    messages.put(True)
    
    with output:
        display("Starting...")
        record = Thread(target=record_microphone)
        record.start()
        
        transcribe = Thread(target=speech_recognition, args=(output,))
        transcribe.start()


# Function to handle the user's speech input
def handle_user_input(user_input):
    global respuesta

    respuesta = conversation.predict(input=user_input)
    generate_speech(respuesta)

# ... (Rest of your code remains the same)

# Function to handle the "Stop" button click
def stop_recording(data):
    global current_audio_path
    global stop_loading_sound

    with output:
        messages.get()
        display("Stopped.")
        # Play the loading sound
        #play(loading_sound)
        
        # Start playing the loading sound in a separate thread
        stop_loading_sound = False
        loading_thread = threading.Thread(target=play_loading_sound_continuously)
        loading_thread.start()

        #display("Transcribed Text:", MENSAJE)

        user_input = MENSAJE

        # Handle the user's input and generate a response
        handle_user_input(user_input)
        
        

        
record_button.on_click(start_recording)
stop_button.on_click(stop_recording)

display(record_button, stop_button, output)


Output()

Button(button_style='success', description='Record', icon='microphone', style=ButtonStyle())

Button(button_style='warning', description='Stop', icon='stop', style=ButtonStyle())

Output()